In [1]:
import numpy as np
import qutip as qt
import qutip.qip.operations as op
from scipy.linalg import expm

from PulseSequence import PulseSequence
from Pulse import Pulse

### Thermal density matrix

In [2]:
pho_th = qt.Qobj(np.diag([1, 0.6, -0.6, -1]), dims=([[2, 2], [2, 2]]))
pho_th

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 1.   0.   0.   0. ]
 [ 0.   0.6  0.   0. ]
 [ 0.   0.  -0.6  0. ]
 [ 0.   0.   0.  -1. ]]

### Helper Functions
See https://github.com/Manvi-Agrawal/Pulse-Sequence-Simulator.

# OPERATORS

In [3]:
x = qt.sigmax()*0.5
y = qt.sigmay()*0.5
z = qt.sigmaz()*0.5
I = qt.qeye(2)

In [4]:
Ix = qt.tensor(x, I)
Iy = qt.tensor(y, I)
Iz = qt.tensor(z, I)

In [5]:
Sx = qt.tensor(I, x)
Sy = qt.tensor(I, y)
Sz = qt.tensor(I, z)

In [6]:
J = 215
tJ = 1/(2*J)
IzSz = qt.tensor(z, z)

In [7]:
print(f"J coupling delay = {np.round(tJ*1000, 3)} ms")

J coupling delay = 2.326 ms


In [8]:
Pulse("+2IzSz", IzSz, (2*np.pi*J)*(tJ) ).apply()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[0.70710678-0.70710678j 0.        +0.j         0.        +0.j
  0.        +0.j        ]
 [0.        +0.j         0.70710678+0.70710678j 0.        +0.j
  0.        +0.j        ]
 [0.        +0.j         0.        +0.j         0.70710678+0.70710678j
  0.        +0.j        ]
 [0.        +0.j         0.        +0.j         0.        +0.j
  0.70710678-0.70710678j]]

In [9]:
# Delay for negative J coupling = 3*tJ; accurate upto global phase
Pulse("-2IzSz", IzSz, (2*np.pi*J)*(3*tJ) ).apply()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[-0.70710678-0.70710678j  0.        +0.j          0.        +0.j
   0.        +0.j        ]
 [ 0.        +0.j         -0.70710678+0.70710678j  0.        +0.j
   0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j         -0.70710678+0.70710678j
   0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j          0.        +0.j
  -0.70710678-0.70710678j]]

We cant implement an $I_z/S_z$ pulse directly.  
Use $ R_z(\theta) = R_x(-pi/2).R_y(\theta).R_x(pi/2)$ 

In [10]:
def Iz_pulse(theta):
    return PulseSequence("Iz(theta)")\
        .add(Pulse("Ix(-pi/2)", Ix, -np.pi/2))\
        .add(Pulse(f"Iy({theta/np.pi}*pi)", Iy, theta))\
        .add(Pulse("Ix(pi/2)", Ix, np.pi/2))

In [11]:
Iz_pulse(np.pi/2).compile().get_operator()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[0.70710678-0.70710678j 0.        +0.j         0.        +0.j
  0.        +0.j        ]
 [0.        +0.j         0.70710678-0.70710678j 0.        +0.j
  0.        +0.j        ]
 [0.        +0.j         0.        +0.j         0.70710678+0.70710678j
  0.        +0.j        ]
 [0.        +0.j         0.        +0.j         0.        +0.j
  0.70710678+0.70710678j]]

In [12]:
(-1j*np.pi/2*Iz).expm()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[0.70710678-0.70710678j 0.        +0.j         0.        +0.j
  0.        +0.j        ]
 [0.        +0.j         0.70710678-0.70710678j 0.        +0.j
  0.        +0.j        ]
 [0.        +0.j         0.        +0.j         0.70710678+0.70710678j
  0.        +0.j        ]
 [0.        +0.j         0.        +0.j         0.        +0.j
  0.70710678+0.70710678j]]

Verified that $R_x(-pi/2).R_y(pi/2).R_x(pi/2) = R_z(pi/2)$ 

In [13]:
Iz_pulse(-np.pi/2).compile().get_operator()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[0.70710678+0.70710678j 0.        +0.j         0.        +0.j
  0.        +0.j        ]
 [0.        +0.j         0.70710678+0.70710678j 0.        +0.j
  0.        +0.j        ]
 [0.        +0.j         0.        +0.j         0.70710678-0.70710678j
  0.        +0.j        ]
 [0.        +0.j         0.        +0.j         0.        +0.j
  0.70710678-0.70710678j]]

In [14]:
(+1j*np.pi/2*Iz).expm()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[0.70710678+0.70710678j 0.        +0.j         0.        +0.j
  0.        +0.j        ]
 [0.        +0.j         0.70710678+0.70710678j 0.        +0.j
  0.        +0.j        ]
 [0.        +0.j         0.        +0.j         0.70710678-0.70710678j
  0.        +0.j        ]
 [0.        +0.j         0.        +0.j         0.        +0.j
  0.70710678-0.70710678j]]

Verified that $R_x(-\pi/2).R_y(-\pi/2).R_x(\pi/2) = R_z(-\pi/2)$ 

In [15]:
def Sz_pulse(theta):
    return PulseSequence("Sz(theta)")\
        .add(Pulse("Sx(-pi/2)", Sx, -np.pi/2))\
        .add(Pulse(f"Sy({theta/np.pi}*pi)", Sy, theta))\
        .add(Pulse("Sx(pi/2)", Sx, np.pi/2))

In [16]:
Sz_pulse(np.pi/2).compile().get_operator()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[0.70710678-0.70710678j 0.        +0.j         0.        +0.j
  0.        +0.j        ]
 [0.        +0.j         0.70710678+0.70710678j 0.        +0.j
  0.        +0.j        ]
 [0.        +0.j         0.        +0.j         0.70710678-0.70710678j
  0.        +0.j        ]
 [0.        +0.j         0.        +0.j         0.        +0.j
  0.70710678+0.70710678j]]

In [17]:
Sz_pulse(np.pi/2).print_sequence()

Sz(theta) Pulse Sequence: Sx(-pi/2) -> Sy(0.5*pi) -> Sx(pi/2)

In [18]:
Sz_pulse(-np.pi/2).compile().get_operator()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[0.70710678+0.70710678j 0.        +0.j         0.        +0.j
  0.        +0.j        ]
 [0.        +0.j         0.70710678-0.70710678j 0.        +0.j
  0.        +0.j        ]
 [0.        +0.j         0.        +0.j         0.70710678+0.70710678j
  0.        +0.j        ]
 [0.        +0.j         0.        +0.j         0.        +0.j
  0.70710678-0.70710678j]]

In [19]:
Sz_pulse(-np.pi/2).print_sequence()


Sz(theta) Pulse Sequence: Sx(-pi/2) -> Sy(-0.5*pi) -> Sx(pi/2)

# Q1: GATES

### PART A: One qubit X

In [20]:
X_i = Pulse("Ix(pi)", Ix, np.pi)
print("Applying X gate to 1H, upto global phase")
np.exp(+1j*np.pi)*X_i.apply()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[0.+0.j 0.+0.j 0.-1.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.-1.j]
 [0.-1.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.-1.j 0.+0.j 0.+0.j]]

In [21]:
X_s = Pulse("Sx(pi)", Sx, np.pi)
print("Applying X gate to 13C, upto global phase")
np.exp(+1j*np.pi)*X_s.apply()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[0.+0.j 0.-1.j 0.+0.j 0.+0.j]
 [0.-1.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.-1.j]
 [0.+0.j 0.+0.j 0.-1.j 0.+0.j]]

### PART B: One qubit Hadamard

#### Pseudo Hadamards

In [22]:
pH_i = Pulse("Iy(-pi/2)", Iy, -np.pi/2)
print(f"Pseudo pH_i pulse : {pH_i.name}")

Pseudo pH_i pulse : Iy(-pi/2)


In [23]:
print(f"Pseudo pH_i operator")
pH_i.apply()

Pseudo pH_i operator


Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[ 0.70710678  0.          0.70710678  0.        ]
 [ 0.          0.70710678  0.          0.70710678]
 [-0.70710678  0.          0.70710678  0.        ]
 [ 0.         -0.70710678  0.          0.70710678]]

In [24]:
pH_i_dag = Pulse("Iy(+pi/2)", Iy, np.pi/2)
print(f"Pseudo pH_i.dag() pulse : {pH_i_dag.name}")

Pseudo pH_i.dag() pulse : Iy(+pi/2)


In [25]:
print(f"Pseudo pH_i.dag() operator")
pH_i_dag.apply()

Pseudo pH_i.dag() operator


Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[ 0.70710678  0.         -0.70710678  0.        ]
 [ 0.          0.70710678  0.         -0.70710678]
 [ 0.70710678  0.          0.70710678  0.        ]
 [ 0.          0.70710678  0.          0.70710678]]

In [26]:
pH_s = Pulse("Sy(-pi/2)", Sy, -np.pi/2)
print(f"Pseudo pH_S pulse : {pH_s.name}")

Pseudo pH_S pulse : Sy(-pi/2)


In [27]:
print(f"Pseudo pH_S operator")
pH_s.apply()

Pseudo pH_S operator


Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[ 0.70710678  0.70710678  0.          0.        ]
 [-0.70710678  0.70710678  0.          0.        ]
 [ 0.          0.          0.70710678  0.70710678]
 [ 0.          0.         -0.70710678  0.70710678]]

In [28]:
pH_s_dag = Pulse("Sy(+pi/2)", Sy, +np.pi/2)
print(f"Pseudo H_S.dag() pulse : {pH_s.name}")

Pseudo H_S.dag() pulse : Sy(-pi/2)


In [29]:
print(f"Pseudo H_S.dag() operator")
pH_s_dag.apply()

Pseudo H_S.dag() operator


Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[ 0.70710678 -0.70710678  0.          0.        ]
 [ 0.70710678  0.70710678  0.          0.        ]
 [ 0.          0.          0.70710678 -0.70710678]
 [ 0.          0.          0.70710678  0.70710678]]

#### Real Hadamards

In [30]:
aH_i = PulseSequence("Actual H_i")\
        .add(Pulse("Iy(+pi/2)", Iy, +np.pi/2))\
        .add(Pulse("Ix(+pi/2)", Ix, +np.pi/2))\
        .add(Pulse("Ix(+pi/2)", Ix, +np.pi/2))\

aH_i.print_sequence()

Actual H_i Pulse Sequence: Iy(+pi/2) -> Ix(+pi/2) -> Ix(+pi/2)

In [31]:
print(f"Actual H_i operator upto global phase")
np.exp(1j*np.pi/2)*aH_i.compile().get_operator()

Actual H_i operator upto global phase


Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 0.70710678  0.          0.70710678  0.        ]
 [ 0.          0.70710678  0.          0.70710678]
 [ 0.70710678  0.         -0.70710678  0.        ]
 [ 0.          0.70710678  0.         -0.70710678]]

In [32]:
aH_s = PulseSequence("Actual H_s")\
        .add(Pulse("Sy(+pi/2)", Sy, +np.pi/2))\
        .add(Pulse("Sx(+pi/2)", Sx, +np.pi/2))\
        .add(Pulse("Sx(+pi/2)", Sx, +np.pi/2))\

aH_s.print_sequence()

Actual H_s Pulse Sequence: Sy(+pi/2) -> Sx(+pi/2) -> Sx(+pi/2)

In [33]:
print(f"Actual H_S operator upto global phase")
np.exp(1j*np.pi/2) * aH_s.compile().get_operator()

Actual H_S operator upto global phase


Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 0.70710678  0.70710678  0.          0.        ]
 [ 0.70710678 -0.70710678  0.          0.        ]
 [ 0.          0.          0.70710678  0.70710678]
 [ 0.          0.          0.70710678 -0.70710678]]

### Part C: Two Qubit Controlled Z

In [34]:
cz_p = PulseSequence("CZ")\
    .add_seq(Iz_pulse(np.pi/2))\
    .add_seq(Sz_pulse(np.pi/2))\
    .add(Pulse("-2IzSz(aka delay(3/2J))", IzSz, 3*np.pi))

In [35]:
print(f"CZ multiplied by phase factor:")
np.exp(1j*np.pi/4)*cz_p.compile().get_operator()

CZ multiplied by phase factor:


Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[-1.  0.  0.  0.]
 [ 0. -1.  0.  0.]
 [ 0.  0. -1.  0.]
 [ 0.  0.  0.  1.]]

In [36]:
cz_p.print_sequence()

CZ Pulse Sequence: Ix(-pi/2) -> Iy(0.5*pi) -> Ix(pi/2) -> Sx(-pi/2) -> Sy(0.5*pi) -> Sx(pi/2) -> -2IzSz(aka delay(3/2J))

### Part D: Two Qubit CNOT

In [37]:
ncx = PulseSequence("Near CNOT")\
    .add(Pulse("Sx(pi/2)", Sx, np.pi/2))\
    .add(Pulse("2IzSz(aka delay(1/2J))", IzSz, np.pi))\
    .add(Pulse("Sy(-pi/2)", Sy, -np.pi/2))

In [38]:
print(f"Near CX multiplied by phase factor: ")
np.exp(-1j*np.pi/4)*ncx.compile().get_operator()

Near CX multiplied by phase factor: 


Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[ 0.-1.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  1.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.-1.j]
 [ 0.+0.j  0.+0.j -1.+0.j  0.+0.j]]

In [39]:
ncx.print_sequence()

Near CNOT Pulse Sequence: Sx(pi/2) -> 2IzSz(aka delay(1/2J)) -> Sy(-pi/2)

In [40]:
cx =  PulseSequence("CNOT")\
    .add(pH_s)\
    .add_seq(cz_p)\
    .add(pH_s_dag)

In [41]:
print("Actual CNOT multiplied by global phase")
np.exp(1j*5*np.pi/4)*cx.compile().get_operator()

Actual CNOT multiplied by global phase


Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]]

In [42]:
cx.print_sequence()

CNOT Pulse Sequence: Sy(-pi/2) -> Ix(-pi/2) -> Iy(0.5*pi) -> Ix(pi/2) -> Sx(-pi/2) -> Sy(0.5*pi) -> Sx(pi/2) -> -2IzSz(aka delay(3/2J)) -> Sy(+pi/2)

# (Q2) Oracles: Deutsch Jozsa

### f1(x) = 0 {Constant 0 function} 
f1(0)=0, f1(1)=0  
Do nothing

In [43]:
f1 = PulseSequence("f1")
f1.print_sequence()

f1 Pulse Sequence: 

### f2(x) = 1 {Constant 1 function} 
f2(0) = 1, f2(1) = 1  
Apply X gate on second qubit(C)


In [44]:
# f1(x) = 1 for x in {0,1}
# Apply X gate on second qubit(C)
f2 = PulseSequence("f2").add(X_s)
f2.print_sequence()

f2 Pulse Sequence: Sx(pi)

In [45]:
f2.compile().get_operator()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[0.+0.j 0.-1.j 0.+0.j 0.+0.j]
 [0.-1.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.-1.j]
 [0.+0.j 0.+0.j 0.-1.j 0.+0.j]]

### f3(x) = x xor y {Balanced function} 
f3(0) = 0, f3(1) = 1  
Apply CX gate, with control on $1^{st}$ qubit and target on $2^{nd}$ qubit.


In [46]:
f3 = PulseSequence("f3").add_seq(cx)
f3.print_sequence()

f3 Pulse Sequence: Sy(-pi/2) -> Ix(-pi/2) -> Iy(0.5*pi) -> Ix(pi/2) -> Sx(-pi/2) -> Sy(0.5*pi) -> Sx(pi/2) -> -2IzSz(aka delay(3/2J)) -> Sy(+pi/2)

In [47]:
print(f"Uf3 is CNOT upto global phase")
(np.exp(-1j*np.pi/4))*f3.compile().get_operator()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[-0.70710678+0.70710678j  0.        +0.j          0.        +0.j
   0.        +0.j        ]
 [ 0.        +0.j         -0.70710678+0.70710678j  0.        +0.j
   0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j          0.        +0.j
  -0.70710678+0.70710678j]
 [ 0.        +0.j          0.        +0.j         -0.70710678+0.70710678j
   0.        +0.j        ]]

### f4(x) = ~x xor y {Balanced function} 
f4(0) = 1, f4(1) = 0  
Apply anti-controlled CX gate, with control on $1^{st}$ qubit and target on $2^{nd}$ qubit. This can be done by applying an $X_{i}$ -> CX -> $X_{i}$ pulse sequence.


In [48]:
f4 = PulseSequence("f4")\
    .add(X_i)\
    .add_seq(cx)\
    .add(X_i)

In [49]:
f4.print_sequence()

f4 Pulse Sequence: Ix(pi) -> Sy(-pi/2) -> Ix(-pi/2) -> Iy(0.5*pi) -> Ix(pi/2) -> Sx(-pi/2) -> Sy(0.5*pi) -> Sx(pi/2) -> -2IzSz(aka delay(3/2J)) -> Sy(+pi/2) -> Ix(pi)

In [50]:
g = qt.basis(2,0)
e = qt.basis(2,1)

In [51]:
print("0-controlled CNOT upto global phase")
np.exp(1j*np.pi/4)*f4.compile().get_operator()

0-controlled CNOT upto global phase


Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]

#### Verification of 0-CNOT

In [52]:
np.exp(1j*np.pi/4)*f4.compile().get_operator()*qt.tensor(g,g)

Quantum object: dims = [[2, 2], [1, 1]], shape = (4, 1), type = ket
Qobj data =
[[0.]
 [1.]
 [0.]
 [0.]]

|00> -> |01>

In [53]:
np.exp(1j*np.pi/4)*f4.compile().get_operator()*qt.tensor(g,e)

Quantum object: dims = [[2, 2], [1, 1]], shape = (4, 1), type = ket
Qobj data =
[[1.]
 [0.]
 [0.]
 [0.]]

|01> -> |00>

In [54]:
np.exp(1j*np.pi/4)*f4.compile().get_operator()*qt.tensor(e,g)

Quantum object: dims = [[2, 2], [1, 1]], shape = (4, 1), type = ket
Qobj data =
[[0.]
 [0.]
 [1.]
 [0.]]

|10> -> |10>

In [55]:
np.exp(1j*np.pi/4)*f4.compile().get_operator()*qt.tensor(e,e)

Quantum object: dims = [[2, 2], [1, 1]], shape = (4, 1), type = ket
Qobj data =
[[0.]
 [0.]
 [0.]
 [1.]]

|11> -> |11>

# (Q3) Deutsch Jozsa(DJ) Algorithm

It is a quantum algorithm to determine if a Boolean function is constant or balanced. 

### Step 1: State Initialisation
We first prepare $|\psi_0\rangle =|00\rangle$ by temporal averaging as done in Lab 3. 

### Step 2: Execute Algorithm
#### Step 2.1 Apply $U_{1}= Iy(+pi/2)Sy(-pi/2)$
After applying $U_1$ to $|\psi_0\rangle =|00\rangle$, we get:
$$
\begin{aligned} 
|\psi_1\rangle=U_1|\psi_0\rangle= \frac{|0\rangle+|1\rangle}{\sqrt{2}}\otimes\frac{|0\rangle-|1\rangle}{\sqrt{2}}\\
=|00\rangle -|01\rangle +|10\rangle-|11\rangle 
\end{aligned}
$$

#### Step 2.2 Apply $U_{fk}$
Here are the four possible boolean functions for two classical bits

|x|f1(x)|f2(x)|f3(x)|f4(x)|
| :---- | :-: | :-: | :-: | :-:
| 0 | 0 | 1 | 0 | 1 |
| 1 | 0 | 1 | 1 | 0 |

Unitary transform $U_{fk}|x\rangle |y\rangle=|x\rangle |f(x)\oplus y\rangle$. The reason can be seen by writing the qubit state after the unitary transform $|\psi_2\rangle=U_f|\psi_1\rangle$.

$$
\begin{aligned}
|\psi_2\rangle&=\frac{1}{2}\{|0\rangle |0\rangle- |0\rangle |f(0)\oplus1\rangle+|1\rangle |f(1)\rangle-|1\rangle |f(1)\oplus1\rangle\}\\
&=\frac{1}{2}\{ (-1)^{f(0)} |0\rangle(|0\rangle-|1\rangle)+(-1)^{f(1)} |1\rangle(|0\rangle-|1\rangle)\}\\
&=\frac{(-1)^{f(0)}|0\rangle+(-1)^{f(1)}|1\rangle}{\sqrt{2}}\otimes\frac{|0\rangle-|1\rangle}{\sqrt{2}}
\end{aligned}
$$

#### Step 2.3 Apply $U_{3}=Iy(-pi/2)Sy(+pi/2)$
Finally, we invert the initial rotations using $U_{3}=Iy(-pi/2)Sy(+pi/2)$
$$

\begin{aligned}
|\psi_3\rangle&=U_3|\psi_2\rangle\\
&= \frac{1}{2}\{(-1)^{f(0)}(|0\rangle-|1\rangle)+(-1)^{f(1)}(|0\rangle+|1\rangle)\}\otimes |0\rangle\\
&= \frac{1}{2} \{((-1)^{f(0)}+(-1)^{f(1)})|0\rangle+(-(-1)^{f(0)}+(-1)^{f(1)})|1\rangle\}\otimes |0\rangle
\end{aligned}
$$

### Step 3: Verify Deutsch Jozsa(DJ) Algorithm

### Case 1: Function is constant: f(0)=f(1)
- Putting $f(0)=f(1)$ in $|\psi_3\rangle$, we get $|\psi_3\rangle= |00\rangle$
- Therefore we should observe same spectra for $U_{f1}$ and $U_{f2}$, which would be different from spectra of $U_{f3}$ and $U_{f4}$.

### Case 2: Function is balanced: f(0)!=f(1)
- Putting $f(0)=(1-f(1))$ in $|\psi_3\rangle$, we get $|\psi_3\rangle= |10\rangle$
- Therefore we should observe same spectra for $U_{f3}$ and $U_{f4}$, which would be different from spectra of $U_{f1}$ and $U_{f2}$.

## Define P1 and P2 operators for Temporal averaging

In [56]:
p0 = PulseSequence("P0")
p0.compile().get_operator()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]

In [57]:
# From lab manual
p1 =  PulseSequence("P1")\
    .add(Pulse("Sx(+pi/2)", Sx, +np.pi/2))\
    .add(Pulse("2IzSz(aka delay(1/2J))", IzSz, +np.pi))\
    .add(Pulse("Sy(+pi/2)", Sy, +np.pi/2))\
    .add(Pulse("Ix(+pi/2)", Ix, +np.pi/2))\
    .add(Pulse("2IzSz(aka delay(1/2J))", IzSz, +np.pi))\
    .add(Pulse("Iy(+pi/2)", Iy, +np.pi/2))

In [58]:
p1.compile().get_operator()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[ 0.+0.j  0.+0.j  0.-1.j  0.+0.j]
 [ 1.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j -1.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.-1.j]]

In [59]:
p1.print_sequence()

P1 Pulse Sequence: Sx(+pi/2) -> 2IzSz(aka delay(1/2J)) -> Sy(+pi/2) -> Ix(+pi/2) -> 2IzSz(aka delay(1/2J)) -> Iy(+pi/2)

In [60]:
# From lab manual
p2 =  PulseSequence("P2")\
    .add(Pulse("Ix(+pi/2)", Ix, +np.pi/2))\
    .add(Pulse("2IzSz(aka delay(1/2J))", IzSz, +np.pi))\
    .add(Pulse("Sx(+pi/2)", Sx, +np.pi/2))\
    .add(Pulse("Iy(+pi/2)", Iy, +np.pi/2))\
    .add(Pulse("2IzSz(aka delay(1/2J))", IzSz, +np.pi))\
    .add(Pulse("Sy(+pi/2)", Sy, +np.pi/2))

In [61]:
p2.compile().get_operator()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[ 0.+0.j  0.-1.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j -1.+0.j  0.+0.j]
 [ 1.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.-1.j]]

In [62]:
p2.print_sequence()

P2 Pulse Sequence: Ix(+pi/2) -> 2IzSz(aka delay(1/2J)) -> Sx(+pi/2) -> Iy(+pi/2) -> 2IzSz(aka delay(1/2J)) -> Sy(+pi/2)

## Step 1 : State Preparation : |00>

### Step 1.1 : P0 simulation for |00>

In [63]:
p0_00 = PulseSequence("P0(00)")\
        .add(X_i)\
        .add(X_s)
p0_00.print_sequence()

P0(00) Pulse Sequence: Ix(pi) -> Sx(pi)

In [64]:
p0_00.compile().get_operator()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 0.  0.  0. -1.]
 [ 0.  0. -1.  0.]
 [ 0. -1.  0.  0.]
 [-1.  0.  0.  0.]]

In [65]:
p0_rho00 = p0_00.compile().evolve_pho(pho_th)
p0_rho00.rho

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[-1.   0.   0.   0. ]
 [ 0.  -0.6  0.   0. ]
 [ 0.   0.   0.6  0. ]
 [ 0.   0.   0.   1. ]]

In [66]:
p0_rho00.h_spectrum()
p0_rho00.c_spectrum()

H spectrum
Peak1 at 4.74 ppm with integral=(-1.6+0j)
Peak2 at 8.18 ppm with integral=(-1.6+0j)
C spectrum
Peak1 @ 66.48 ppm with integral=(-0.4+0j)
Peak2 @ 80.19 ppm with integral=(-0.4+0j)


#### Step 1.2 : P1 simulation for |00>

In [67]:
p1_00 = PulseSequence("P1(00)")\
        .add_seq(p1)\
        .add(X_i)\
        .add(X_s)

In [68]:
p1_00.print_sequence()

P1(00) Pulse Sequence: Sx(+pi/2) -> 2IzSz(aka delay(1/2J)) -> Sy(+pi/2) -> Ix(+pi/2) -> 2IzSz(aka delay(1/2J)) -> Iy(+pi/2) -> Ix(pi) -> Sx(pi)

In [69]:
p1_00.compile().get_operator()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[ 0.+0.j  0.+0.j  0.+0.j  0.+1.j]
 [ 0.+0.j  1.+0.j  0.+0.j  0.+0.j]
 [-1.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+1.j  0.+0.j]]

In [70]:
p1_rho00 = p1_00.compile().evolve_pho(pho_th)
p1_rho00.rho

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[-1.   0.   0.   0. ]
 [ 0.   0.6  0.   0. ]
 [ 0.   0.   1.   0. ]
 [ 0.   0.   0.  -0.6]]

In [71]:
p1_rho00.h_spectrum()
p1_rho00.c_spectrum()

H spectrum
Peak1 at 4.74 ppm with integral=(-2+0j)
Peak2 at 8.18 ppm with integral=(1.2+0j)
C spectrum
Peak1 @ 66.48 ppm with integral=(-1.6+0j)
Peak2 @ 80.19 ppm with integral=(1.6+0j)


### Step 1.3 : P2 simulation for |00>

In [72]:
p2_00 = PulseSequence("P2(00)")\
        .add_seq(p2)\
        .add(X_i)\
        .add(X_s)

In [73]:
p2_00.print_sequence()

P2(00) Pulse Sequence: Ix(+pi/2) -> 2IzSz(aka delay(1/2J)) -> Sx(+pi/2) -> Iy(+pi/2) -> 2IzSz(aka delay(1/2J)) -> Sy(+pi/2) -> Ix(pi) -> Sx(pi)

In [74]:
p2_00.compile().get_operator()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[ 0.+0.j  0.+0.j  0.+0.j  0.+1.j]
 [-1.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  1.+0.j  0.+0.j]
 [ 0.+0.j  0.+1.j  0.+0.j  0.+0.j]]

In [75]:
p2_rho00 = p2_00.compile().evolve_pho(pho_th)
p2_rho00.rho

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[-1.   0.   0.   0. ]
 [ 0.   1.   0.   0. ]
 [ 0.   0.  -0.6  0. ]
 [ 0.   0.   0.   0.6]]

In [76]:
p2_rho00.h_spectrum()
p2_rho00.c_spectrum()

H spectrum
Peak1 at 4.74 ppm with integral=(-0.4+0j)
Peak2 at 8.18 ppm with integral=(0.4+0j)
C spectrum
Peak1 @ 66.48 ppm with integral=(-2+0j)
Peak2 @ 80.19 ppm with integral=(-1.2+0j)


### Step 1.4 : Calculate temporal average for |00>

In [77]:
(p0_rho00.rho + p1_rho00.rho + p2_rho00.rho)/3

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[-1.          0.          0.          0.        ]
 [ 0.          0.33333333  0.          0.        ]
 [ 0.          0.          0.33333333  0.        ]
 [ 0.          0.          0.          0.33333333]]

## Step 2 : Execute DJ Algorithm : |00>
- Case A: $U_{f1}(0)=0, \ U_{f1}(1)=0$
- Case B: $U_{f2}(0)=1, \ U_{f2}(1)=1$
- Case C: $U_{f3}(0)=0, \ U_{f3}(1)=1$
- Case D: $U_{f4}(0)=1, \ U_{f4}(1)=0$

### Step 2A : $U_{f1}(0)=0, \ U_{f1}(1)=0$

#### Step 2A.1 : $U_{f1}$ P0 simulation for |00>

In [78]:
p0_dj_uf1 = PulseSequence("DJ-Uf1(00)")\
        .add_seq(p0_00)\
        .add(Pulse("Iy(+pi/2)", Iy, +np.pi/2))\
        .add(Pulse("Sy(-pi/2)", Sy, -np.pi/2))\
        .add_seq(f1)\
        .add(Pulse("Iy(-pi/2)", Iy, -np.pi/2))\
        .add(Pulse("Sy(+pi/2)", Sy, +np.pi/2))\
        
p0_dj_uf1.print_sequence()

DJ-Uf1(00) Pulse Sequence: Ix(pi) -> Sx(pi) -> Iy(+pi/2) -> Sy(-pi/2) -> Iy(-pi/2) -> Sy(+pi/2)

In [79]:
p0_dj_uf1.compile().get_operator()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 0.  0.  0. -1.]
 [ 0.  0. -1.  0.]
 [ 0. -1.  0.  0.]
 [-1.  0.  0.  0.]]

In [80]:
p0_dj_uf1 = p0_dj_uf1.compile().evolve_pho(pho_th)
p0_dj_uf1.rho

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[-1.   0.   0.   0. ]
 [ 0.  -0.6  0.   0. ]
 [ 0.   0.   0.6  0. ]
 [ 0.   0.   0.   1. ]]

In [81]:
p0_dj_uf1.h_spectrum()
p0_dj_uf1.c_spectrum()

H spectrum
Peak1 at 4.74 ppm with integral=(-1.6+0j)
Peak2 at 8.18 ppm with integral=(-1.6+0j)
C spectrum
Peak1 @ 66.48 ppm with integral=(-0.4+0j)
Peak2 @ 80.19 ppm with integral=(-0.4+0j)


#### Step 2A.2 : $U_{f1}$ P1 simulation for |00>

In [82]:
p1_dj_uf1 = PulseSequence("P1 :: DJ-Uf1(00)")\
        .add_seq(p1_00)\
        .add(Pulse("Iy(+pi/2)", Iy, +np.pi/2))\
        .add(Pulse("Sy(-pi/2)", Sy, -np.pi/2))\
        .add_seq(f1)\
        .add(Pulse("Iy(-pi/2)", Iy, -np.pi/2))\
        .add(Pulse("Sy(+pi/2)", Sy, +np.pi/2))\
        
p1_dj_uf1.print_sequence()

P1 :: DJ-Uf1(00) Pulse Sequence: Sx(+pi/2) -> 2IzSz(aka delay(1/2J)) -> Sy(+pi/2) -> Ix(+pi/2) -> 2IzSz(aka delay(1/2J)) -> Iy(+pi/2) -> Ix(pi) -> Sx(pi) -> Iy(+pi/2) -> Sy(-pi/2) -> Iy(-pi/2) -> Sy(+pi/2)

In [83]:
p1_dj_uf1.compile().get_operator()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[ 0.+0.j  0.+0.j  0.+0.j  0.+1.j]
 [ 0.+0.j  1.+0.j  0.+0.j  0.+0.j]
 [-1.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+1.j  0.+0.j]]

In [84]:
p1_dj_uf1 = p1_dj_uf1.compile().evolve_pho(pho_th)
p1_dj_uf1.rho

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[-1.   0.   0.   0. ]
 [ 0.   0.6  0.   0. ]
 [ 0.   0.   1.   0. ]
 [ 0.   0.   0.  -0.6]]

In [85]:
p1_dj_uf1.h_spectrum()
p1_dj_uf1.c_spectrum()

H spectrum
Peak1 at 4.74 ppm with integral=(-2+0j)
Peak2 at 8.18 ppm with integral=(1.2+0j)
C spectrum
Peak1 @ 66.48 ppm with integral=(-1.6+0j)
Peak2 @ 80.19 ppm with integral=(1.6+0j)


#### Step 2A.3 : $U_{f1}$ P2 simulation for |00>

In [86]:
p2_dj_uf1 = PulseSequence("P2 :: DJ-Uf1(00)")\
        .add_seq(p2_00)\
        .add(Pulse("Iy(+pi/2)", Iy, +np.pi/2))\
        .add(Pulse("Sy(-pi/2)", Sy, -np.pi/2))\
        .add_seq(f1)\
        .add(Pulse("Iy(-pi/2)", Iy, -np.pi/2))\
        .add(Pulse("Sy(+pi/2)", Sy, +np.pi/2))\
        
p2_dj_uf1.print_sequence()

P2 :: DJ-Uf1(00) Pulse Sequence: Ix(+pi/2) -> 2IzSz(aka delay(1/2J)) -> Sx(+pi/2) -> Iy(+pi/2) -> 2IzSz(aka delay(1/2J)) -> Sy(+pi/2) -> Ix(pi) -> Sx(pi) -> Iy(+pi/2) -> Sy(-pi/2) -> Iy(-pi/2) -> Sy(+pi/2)

In [87]:
p2_dj_uf1.compile().get_operator()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[ 0.+0.j  0.+0.j  0.+0.j  0.+1.j]
 [-1.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  1.+0.j  0.+0.j]
 [ 0.+0.j  0.+1.j  0.+0.j  0.+0.j]]

In [88]:
p2_dj_uf1 = p2_dj_uf1.compile().evolve_pho(pho_th)
p2_dj_uf1.rho

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[-1.   0.   0.   0. ]
 [ 0.   1.   0.   0. ]
 [ 0.   0.  -0.6  0. ]
 [ 0.   0.   0.   0.6]]

In [89]:
p2_dj_uf1.h_spectrum()
p2_dj_uf1.c_spectrum()

H spectrum
Peak1 at 4.74 ppm with integral=(-0.4+0j)
Peak2 at 8.18 ppm with integral=(0.4+0j)
C spectrum
Peak1 @ 66.48 ppm with integral=(-2+0j)
Peak2 @ 80.19 ppm with integral=(-1.2+0j)


#### Step 2A.4 : Temporal averaging for DJ $U_{f1}$ |00>

In [90]:
(p0_dj_uf1.rho + p1_dj_uf1.rho + p2_dj_uf1.rho)/3

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[-1.          0.          0.          0.        ]
 [ 0.          0.33333333  0.          0.        ]
 [ 0.          0.          0.33333333  0.        ]
 [ 0.          0.          0.          0.33333333]]

### Step 2B : $U_{f2}$

#### Step 2B.1 : $U_{f2}$ P0 simulation for |00>

In [91]:
p0_dj_uf2 = PulseSequence("DJ-Uf2(00)")\
        .add_seq(p0_00)\
        .add(Pulse("Iy(+pi/2)", Iy, +np.pi/2))\
        .add(Pulse("Sy(-pi/2)", Sy, -np.pi/2))\
        .add_seq(f2)\
        .add(Pulse("Iy(-pi/2)", Iy, -np.pi/2))\
        .add(Pulse("Sy(+pi/2)", Sy, +np.pi/2))\
        
p0_dj_uf2.print_sequence()

DJ-Uf2(00) Pulse Sequence: Ix(pi) -> Sx(pi) -> Iy(+pi/2) -> Sy(-pi/2) -> Sx(pi) -> Iy(-pi/2) -> Sy(+pi/2)

In [92]:
p0_dj_uf2.compile().get_operator()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[0.+0.j 0.+0.j 0.+0.j 0.-1.j]
 [0.+0.j 0.+0.j 0.+1.j 0.+0.j]
 [0.+0.j 0.-1.j 0.+0.j 0.+0.j]
 [0.+1.j 0.+0.j 0.+0.j 0.+0.j]]

In [93]:
p0_dj_uf2 = p0_dj_uf2.compile().evolve_pho(pho_th)
p0_dj_uf2.rho

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[-1.   0.   0.   0. ]
 [ 0.  -0.6  0.   0. ]
 [ 0.   0.   0.6  0. ]
 [ 0.   0.   0.   1. ]]

In [94]:
p0_dj_uf2.h_spectrum()
p0_dj_uf2.c_spectrum()

H spectrum
Peak1 at 4.74 ppm with integral=(-1.6+0j)
Peak2 at 8.18 ppm with integral=(-1.6+0j)
C spectrum
Peak1 @ 66.48 ppm with integral=(-0.4+0j)
Peak2 @ 80.19 ppm with integral=(-0.4+0j)


#### Step 2B.2 : $U_{f2}$ P1 simulation for |00>

In [95]:
p1_dj_uf2 = PulseSequence("P1 :: DJ-Uf2(00)")\
        .add_seq(p1_00)\
        .add(Pulse("Iy(+pi/2)", Iy, +np.pi/2))\
        .add(Pulse("Sy(-pi/2)", Sy, -np.pi/2))\
        .add_seq(f2)\
        .add(Pulse("Iy(-pi/2)", Iy, -np.pi/2))\
        .add(Pulse("Sy(+pi/2)", Sy, +np.pi/2))\
        
p1_dj_uf2.print_sequence()

P1 :: DJ-Uf2(00) Pulse Sequence: Sx(+pi/2) -> 2IzSz(aka delay(1/2J)) -> Sy(+pi/2) -> Ix(+pi/2) -> 2IzSz(aka delay(1/2J)) -> Iy(+pi/2) -> Ix(pi) -> Sx(pi) -> Iy(+pi/2) -> Sy(-pi/2) -> Sx(pi) -> Iy(-pi/2) -> Sy(+pi/2)

In [96]:
p1_dj_uf2.compile().get_operator()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[ 0.+0.j  0.+0.j  0.+0.j -1.+0.j]
 [ 0.+0.j  0.-1.j  0.+0.j  0.+0.j]
 [ 0.-1.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  1.+0.j  0.+0.j]]

In [97]:
p1_dj_uf2 = p1_dj_uf2.compile().evolve_pho(pho_th)
p1_dj_uf2.rho

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[-1.   0.   0.   0. ]
 [ 0.   0.6  0.   0. ]
 [ 0.   0.   1.   0. ]
 [ 0.   0.   0.  -0.6]]

In [98]:
p1_dj_uf2.h_spectrum()
p1_dj_uf2.c_spectrum()

H spectrum
Peak1 at 4.74 ppm with integral=(-2+0j)
Peak2 at 8.18 ppm with integral=(1.2+0j)
C spectrum
Peak1 @ 66.48 ppm with integral=(-1.6+0j)
Peak2 @ 80.19 ppm with integral=(1.6+0j)


#### Step 2B.3 : $U_{f2}$ P2 simulation for |00>

In [99]:
p2_dj_uf2 = PulseSequence("P2 :: DJ-Uf2(00)")\
        .add_seq(p2_00)\
        .add(Pulse("Iy(+pi/2)", Iy, +np.pi/2))\
        .add(Pulse("Sy(-pi/2)", Sy, -np.pi/2))\
        .add_seq(f2)\
        .add(Pulse("Iy(-pi/2)", Iy, -np.pi/2))\
        .add(Pulse("Sy(+pi/2)", Sy, +np.pi/2))\
        

p2_dj_uf2.print_sequence()

P2 :: DJ-Uf2(00) Pulse Sequence: Ix(+pi/2) -> 2IzSz(aka delay(1/2J)) -> Sx(+pi/2) -> Iy(+pi/2) -> 2IzSz(aka delay(1/2J)) -> Sy(+pi/2) -> Ix(pi) -> Sx(pi) -> Iy(+pi/2) -> Sy(-pi/2) -> Sx(pi) -> Iy(-pi/2) -> Sy(+pi/2)

In [100]:
p2_dj_uf2.compile().get_operator()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[ 0.+0.j  0.+0.j  0.+0.j -1.+0.j]
 [ 0.+1.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+1.j  0.+0.j]
 [ 0.+0.j  1.+0.j  0.+0.j  0.+0.j]]

In [101]:
p2_dj_uf2 = p2_dj_uf2.compile().evolve_pho(pho_th)
p2_dj_uf2.rho

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[-1.   0.   0.   0. ]
 [ 0.   1.   0.   0. ]
 [ 0.   0.  -0.6  0. ]
 [ 0.   0.   0.   0.6]]

In [102]:
p2_dj_uf2.h_spectrum()
p2_dj_uf2.c_spectrum()

H spectrum
Peak1 at 4.74 ppm with integral=(-0.4+0j)
Peak2 at 8.18 ppm with integral=(0.4+0j)
C spectrum
Peak1 @ 66.48 ppm with integral=(-2+0j)
Peak2 @ 80.19 ppm with integral=(-1.2+0j)


#### Step 2B.4 : Temporal averaging for DJ $U_{f2}$ |00>

In [103]:
(p0_dj_uf2.rho + p1_dj_uf2.rho + p2_dj_uf2.rho)/3

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[-1.          0.          0.          0.        ]
 [ 0.          0.33333333  0.          0.        ]
 [ 0.          0.          0.33333333  0.        ]
 [ 0.          0.          0.          0.33333333]]

In [104]:
# %reset_selective dj*

### Step 2C : $U_{f3}$

#### Step 2C.1 : $U_{f3}$ P0 simulation for |00>

In [105]:
p0_dj_uf3 = PulseSequence("DJ-Uf3(00)")\
        .add_seq(p0_00)\
        .add(Pulse("Iy(+pi/2)", Iy, +np.pi/2))\
        .add(Pulse("Sy(-pi/2)", Sy, -np.pi/2))\
        .add_seq(f3)\
        .add(Pulse("Iy(-pi/2)", Iy, -np.pi/2))\
        .add(Pulse("Sy(+pi/2)", Sy, +np.pi/2))\
        
p0_dj_uf3.print_sequence()

DJ-Uf3(00) Pulse Sequence: Ix(pi) -> Sx(pi) -> Iy(+pi/2) -> Sy(-pi/2) -> Sy(-pi/2) -> Ix(-pi/2) -> Iy(0.5*pi) -> Ix(pi/2) -> Sx(-pi/2) -> Sy(0.5*pi) -> Sx(pi/2) -> -2IzSz(aka delay(3/2J)) -> Sy(+pi/2) -> Iy(-pi/2) -> Sy(+pi/2)

In [106]:
p0_dj_uf3.compile().get_operator()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[ 0.        +0.j         -0.70710678+0.70710678j  0.        +0.j
   0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j          0.70710678-0.70710678j
   0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j          0.        +0.j
  -0.70710678+0.70710678j]
 [ 0.70710678-0.70710678j  0.        +0.j          0.        +0.j
   0.        +0.j        ]]

In [107]:
p0_dj_uf3 = p0_dj_uf3.compile().evolve_pho(pho_th)
p0_dj_uf3.rho

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 0.6  0.   0.   0. ]
 [ 0.  -0.6  0.   0. ]
 [ 0.   0.  -1.   0. ]
 [ 0.   0.   0.   1. ]]

In [108]:
p0_dj_uf3.h_spectrum()
p0_dj_uf3.c_spectrum()

H spectrum
Peak1 at 4.74 ppm with integral=(1.6+0j)
Peak2 at 8.18 ppm with integral=(-1.6+0j)
C spectrum
Peak1 @ 66.48 ppm with integral=(1.2+0j)
Peak2 @ 80.19 ppm with integral=(-2+0j)


#### Step 2C.2 : $U_{f3}$ P1 simulation for |00>

In [109]:
p1_dj_uf3 = PulseSequence("P1 :: DJ-Uf3(00)")\
        .add_seq(p1_00)\
        .add(Pulse("Iy(+pi/2)", Iy, +np.pi/2))\
        .add(Pulse("Sy(-pi/2)", Sy, -np.pi/2))\
        .add_seq(f3)\
        .add(Pulse("Iy(-pi/2)", Iy, -np.pi/2))\
        .add(Pulse("Sy(+pi/2)", Sy, +np.pi/2))\
        
p1_dj_uf3.print_sequence()

P1 :: DJ-Uf3(00) Pulse Sequence: Sx(+pi/2) -> 2IzSz(aka delay(1/2J)) -> Sy(+pi/2) -> Ix(+pi/2) -> 2IzSz(aka delay(1/2J)) -> Iy(+pi/2) -> Ix(pi) -> Sx(pi) -> Iy(+pi/2) -> Sy(-pi/2) -> Sy(-pi/2) -> Ix(-pi/2) -> Iy(0.5*pi) -> Ix(pi/2) -> Sx(-pi/2) -> Sy(0.5*pi) -> Sx(pi/2) -> -2IzSz(aka delay(3/2J)) -> Sy(+pi/2) -> Iy(-pi/2) -> Sy(+pi/2)

In [110]:
p1_dj_uf3.compile().get_operator()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[-0.70710678+0.70710678j  0.        +0.j          0.        +0.j
   0.        +0.j        ]
 [ 0.        +0.j         -0.70710678+0.70710678j  0.        +0.j
   0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j          0.        +0.j
   0.70710678+0.70710678j]
 [ 0.        +0.j          0.        +0.j         -0.70710678-0.70710678j
   0.        +0.j        ]]

In [111]:
p1_dj_uf3 = p1_dj_uf3.compile().evolve_pho(pho_th)
p1_dj_uf3.rho

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 1.   0.   0.   0. ]
 [ 0.   0.6  0.   0. ]
 [ 0.   0.  -1.   0. ]
 [ 0.   0.   0.  -0.6]]

In [112]:
p1_dj_uf3.h_spectrum()
p1_dj_uf3.c_spectrum()

H spectrum
Peak1 at 4.74 ppm with integral=(2+0j)
Peak2 at 8.18 ppm with integral=(1.2+0j)
C spectrum
Peak1 @ 66.48 ppm with integral=(0.4+0j)
Peak2 @ 80.19 ppm with integral=(-0.4+0j)


#### Step 2C.3 : $U_{f3}$ P2 simulation for |00>

In [113]:
p2_dj_uf3 = PulseSequence("P2 :: DJ-Uf3(00)")\
        .add_seq(p2_00)\
        .add(Pulse("Iy(+pi/2)", Iy, +np.pi/2))\
        .add(Pulse("Sy(-pi/2)", Sy, -np.pi/2))\
        .add_seq(f3)\
        .add(Pulse("Iy(-pi/2)", Iy, -np.pi/2))\
        .add(Pulse("Sy(+pi/2)", Sy, +np.pi/2))\
        

p2_dj_uf3.print_sequence()

P2 :: DJ-Uf3(00) Pulse Sequence: Ix(+pi/2) -> 2IzSz(aka delay(1/2J)) -> Sx(+pi/2) -> Iy(+pi/2) -> 2IzSz(aka delay(1/2J)) -> Sy(+pi/2) -> Ix(pi) -> Sx(pi) -> Iy(+pi/2) -> Sy(-pi/2) -> Sy(-pi/2) -> Ix(-pi/2) -> Iy(0.5*pi) -> Ix(pi/2) -> Sx(-pi/2) -> Sy(0.5*pi) -> Sx(pi/2) -> -2IzSz(aka delay(3/2J)) -> Sy(+pi/2) -> Iy(-pi/2) -> Sy(+pi/2)

In [114]:
p2_dj_uf3.compile().get_operator()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[ 0.        +0.j          0.        +0.j          0.70710678-0.70710678j
   0.        +0.j        ]
 [ 0.70710678-0.70710678j  0.        +0.j          0.        +0.j
   0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j          0.        +0.j
   0.70710678+0.70710678j]
 [ 0.        +0.j         -0.70710678-0.70710678j  0.        +0.j
   0.        +0.j        ]]

In [115]:
p2_dj_uf3 = p2_dj_uf3.compile().evolve_pho(pho_th)
p2_dj_uf3.rho

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[-0.6  0.   0.   0. ]
 [ 0.   1.   0.   0. ]
 [ 0.   0.  -1.   0. ]
 [ 0.   0.   0.   0.6]]

In [116]:
p2_dj_uf3.h_spectrum()
p2_dj_uf3.c_spectrum()

H spectrum
Peak1 at 4.74 ppm with integral=(0.4+0j)
Peak2 at 8.18 ppm with integral=(0.4+0j)
C spectrum
Peak1 @ 66.48 ppm with integral=(-1.6+0j)
Peak2 @ 80.19 ppm with integral=(-1.6+0j)


#### Step 2C.4 : Temporal averaging for DJ $U_{f3}$ |00>

In [117]:
(p0_dj_uf3.rho + p1_dj_uf3.rho + p2_dj_uf3.rho)/3

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 0.33333333  0.          0.          0.        ]
 [ 0.          0.33333333  0.          0.        ]
 [ 0.          0.         -1.          0.        ]
 [ 0.          0.          0.          0.33333333]]

### Step 2D : $U_{f4}$

#### Step 2D.1 : $U_{f4}$ P0 simulation for |00>

In [118]:
p0_dj_uf4 = PulseSequence("DJ-Uf4(00)")\
        .add_seq(p0_00)\
        .add(Pulse("Iy(+pi/2)", Iy, +np.pi/2))\
        .add(Pulse("Sy(-pi/2)", Sy, -np.pi/2))\
        .add_seq(f4)\
        .add(Pulse("Iy(-pi/2)", Iy, -np.pi/2))\
        .add(Pulse("Sy(+pi/2)", Sy, +np.pi/2))\
        
p0_dj_uf4.print_sequence()

DJ-Uf4(00) Pulse Sequence: Ix(pi) -> Sx(pi) -> Iy(+pi/2) -> Sy(-pi/2) -> Ix(pi) -> Sy(-pi/2) -> Ix(-pi/2) -> Iy(0.5*pi) -> Ix(pi/2) -> Sx(-pi/2) -> Sy(0.5*pi) -> Sx(pi/2) -> -2IzSz(aka delay(3/2J)) -> Sy(+pi/2) -> Ix(pi) -> Iy(-pi/2) -> Sy(+pi/2)

In [119]:
p0_dj_uf4.compile().get_operator()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[ 0.        +0.j         -0.70710678+0.70710678j  0.        +0.j
   0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j         -0.70710678+0.70710678j
   0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j          0.        +0.j
  -0.70710678+0.70710678j]
 [-0.70710678+0.70710678j  0.        +0.j          0.        +0.j
   0.        +0.j        ]]

In [120]:
p0_dj_uf4 = p0_dj_uf4.compile().evolve_pho(pho_th)
p0_dj_uf4.rho

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 0.6  0.   0.   0. ]
 [ 0.  -0.6  0.   0. ]
 [ 0.   0.  -1.   0. ]
 [ 0.   0.   0.   1. ]]

In [121]:
p0_dj_uf4.h_spectrum()
p0_dj_uf4.c_spectrum()

H spectrum
Peak1 at 4.74 ppm with integral=(1.6+0j)
Peak2 at 8.18 ppm with integral=(-1.6+0j)
C spectrum
Peak1 @ 66.48 ppm with integral=(1.2+0j)
Peak2 @ 80.19 ppm with integral=(-2+0j)


#### Step 2D.2 : $U_{f4}$ P1 simulation for |00>

In [122]:
p1_dj_uf4 = PulseSequence("P1 :: DJ-Uf4(00)")\
        .add_seq(p1_00)\
        .add(Pulse("Iy(+pi/2)", Iy, +np.pi/2))\
        .add(Pulse("Sy(-pi/2)", Sy, -np.pi/2))\
        .add_seq(f4)\
        .add(Pulse("Iy(-pi/2)", Iy, -np.pi/2))\
        .add(Pulse("Sy(+pi/2)", Sy, +np.pi/2))\
        
p1_dj_uf4.print_sequence()

P1 :: DJ-Uf4(00) Pulse Sequence: Sx(+pi/2) -> 2IzSz(aka delay(1/2J)) -> Sy(+pi/2) -> Ix(+pi/2) -> 2IzSz(aka delay(1/2J)) -> Iy(+pi/2) -> Ix(pi) -> Sx(pi) -> Iy(+pi/2) -> Sy(-pi/2) -> Ix(pi) -> Sy(-pi/2) -> Ix(-pi/2) -> Iy(0.5*pi) -> Ix(pi/2) -> Sx(-pi/2) -> Sy(0.5*pi) -> Sx(pi/2) -> -2IzSz(aka delay(3/2J)) -> Sy(+pi/2) -> Ix(pi) -> Iy(-pi/2) -> Sy(+pi/2)

In [123]:
p1_dj_uf4.compile().get_operator()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[-0.70710678+0.70710678j  0.        +0.j          0.        +0.j
   0.        +0.j        ]
 [ 0.        +0.j          0.70710678-0.70710678j  0.        +0.j
   0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j          0.        +0.j
   0.70710678+0.70710678j]
 [ 0.        +0.j          0.        +0.j          0.70710678+0.70710678j
   0.        +0.j        ]]

In [124]:
p1_dj_uf4 = p1_dj_uf4.compile().evolve_pho(pho_th)
p1_dj_uf4.rho

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 1.   0.   0.   0. ]
 [ 0.   0.6  0.   0. ]
 [ 0.   0.  -1.   0. ]
 [ 0.   0.   0.  -0.6]]

In [125]:
p1_dj_uf4.h_spectrum()
p1_dj_uf4.c_spectrum()

H spectrum
Peak1 at 4.74 ppm with integral=(2+0j)
Peak2 at 8.18 ppm with integral=(1.2+0j)
C spectrum
Peak1 @ 66.48 ppm with integral=(0.4+0j)
Peak2 @ 80.19 ppm with integral=(-0.4+0j)


#### Step 2D.3 : $U_{f4}$ P2 simulation for |00>

In [126]:
p2_dj_uf4 = PulseSequence("P2 :: DJ-Uf4(00)")\
        .add_seq(p2_00)\
        .add(Pulse("Iy(+pi/2)", Iy, +np.pi/2))\
        .add(Pulse("Sy(-pi/2)", Sy, -np.pi/2))\
        .add_seq(f4)\
        .add(Pulse("Iy(-pi/2)", Iy, -np.pi/2))\
        .add(Pulse("Sy(+pi/2)", Sy, +np.pi/2))\
        

p2_dj_uf4.print_sequence()

P2 :: DJ-Uf4(00) Pulse Sequence: Ix(+pi/2) -> 2IzSz(aka delay(1/2J)) -> Sx(+pi/2) -> Iy(+pi/2) -> 2IzSz(aka delay(1/2J)) -> Sy(+pi/2) -> Ix(pi) -> Sx(pi) -> Iy(+pi/2) -> Sy(-pi/2) -> Ix(pi) -> Sy(-pi/2) -> Ix(-pi/2) -> Iy(0.5*pi) -> Ix(pi/2) -> Sx(-pi/2) -> Sy(0.5*pi) -> Sx(pi/2) -> -2IzSz(aka delay(3/2J)) -> Sy(+pi/2) -> Ix(pi) -> Iy(-pi/2) -> Sy(+pi/2)

In [127]:
p2_dj_uf4.compile().get_operator()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[ 0.        +0.j          0.        +0.j          0.70710678-0.70710678j
   0.        +0.j        ]
 [-0.70710678+0.70710678j  0.        +0.j          0.        +0.j
   0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j          0.        +0.j
   0.70710678+0.70710678j]
 [ 0.        +0.j          0.70710678+0.70710678j  0.        +0.j
   0.        +0.j        ]]

In [128]:
p2_dj_uf4 = p2_dj_uf4.compile().evolve_pho(pho_th)
p2_dj_uf4.rho

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[-0.6  0.   0.   0. ]
 [ 0.   1.   0.   0. ]
 [ 0.   0.  -1.   0. ]
 [ 0.   0.   0.   0.6]]

In [129]:
p2_dj_uf4.h_spectrum()
p2_dj_uf4.c_spectrum()

H spectrum
Peak1 at 4.74 ppm with integral=(0.4+0j)
Peak2 at 8.18 ppm with integral=(0.4+0j)
C spectrum
Peak1 @ 66.48 ppm with integral=(-1.6+0j)
Peak2 @ 80.19 ppm with integral=(-1.6+0j)


#### Step 2D.4 : Temporal averaging for DJ $U_{f4}$ |00>

In [130]:
(p0_dj_uf4.rho + p1_dj_uf4.rho + p2_dj_uf4.rho)/3

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 0.33333333  0.          0.          0.        ]
 [ 0.          0.33333333  0.          0.        ]
 [ 0.          0.         -1.          0.        ]
 [ 0.          0.          0.          0.33333333]]

## Step 3: Verify Deutsch Jozsa(DJ) Algorithm

### Step 3.1 : Verify constant function(f(0)=f(1))

In [131]:
print("---------------------------------------------------")
print("P0-Uf1(00) Spectrum")
print("---------------------------------------------------")
p0_dj_uf1.h_spectrum()
p0_dj_uf1.c_spectrum()

print("\n")
print("---------------------------------------------------")
print("P0-Uf2(00) Spectrum")
print("---------------------------------------------------")
p0_dj_uf2.h_spectrum()
p0_dj_uf2.c_spectrum()

---------------------------------------------------
P0-Uf1(00) Spectrum
---------------------------------------------------
H spectrum
Peak1 at 4.74 ppm with integral=(-1.6+0j)
Peak2 at 8.18 ppm with integral=(-1.6+0j)
C spectrum
Peak1 @ 66.48 ppm with integral=(-0.4+0j)
Peak2 @ 80.19 ppm with integral=(-0.4+0j)


---------------------------------------------------
P0-Uf2(00) Spectrum
---------------------------------------------------
H spectrum
Peak1 at 4.74 ppm with integral=(-1.6+0j)
Peak2 at 8.18 ppm with integral=(-1.6+0j)
C spectrum
Peak1 @ 66.48 ppm with integral=(-0.4+0j)
Peak2 @ 80.19 ppm with integral=(-0.4+0j)


In [132]:
print("---------------------------------------------------")
print("P1-Uf1(00) Spectrum")
print("---------------------------------------------------")
p1_dj_uf1.h_spectrum()
p1_dj_uf1.c_spectrum()

print("\n")
print("---------------------------------------------------")
print("\nP1-Uf2(00) Spectrum")
print("---------------------------------------------------")
p1_dj_uf2.h_spectrum()
p1_dj_uf2.c_spectrum()

---------------------------------------------------
P1-Uf1(00) Spectrum
---------------------------------------------------
H spectrum
Peak1 at 4.74 ppm with integral=(-2+0j)
Peak2 at 8.18 ppm with integral=(1.2+0j)
C spectrum
Peak1 @ 66.48 ppm with integral=(-1.6+0j)
Peak2 @ 80.19 ppm with integral=(1.6+0j)


---------------------------------------------------

P1-Uf2(00) Spectrum
---------------------------------------------------
H spectrum
Peak1 at 4.74 ppm with integral=(-2+0j)
Peak2 at 8.18 ppm with integral=(1.2+0j)
C spectrum
Peak1 @ 66.48 ppm with integral=(-1.6+0j)
Peak2 @ 80.19 ppm with integral=(1.6+0j)


In [133]:
print("---------------------------------------------------")
print("P2-Uf1(00) Spectrum")
print("---------------------------------------------------")
p2_dj_uf1.h_spectrum()
p2_dj_uf1.c_spectrum()

print("\n")
print("---------------------------------------------------")
print("\nP2-Uf2(00) Spectrum")
print("---------------------------------------------------")
p2_dj_uf2.h_spectrum()
p2_dj_uf2.c_spectrum()

---------------------------------------------------
P2-Uf1(00) Spectrum
---------------------------------------------------
H spectrum
Peak1 at 4.74 ppm with integral=(-0.4+0j)
Peak2 at 8.18 ppm with integral=(0.4+0j)
C spectrum
Peak1 @ 66.48 ppm with integral=(-2+0j)
Peak2 @ 80.19 ppm with integral=(-1.2+0j)


---------------------------------------------------

P2-Uf2(00) Spectrum
---------------------------------------------------
H spectrum
Peak1 at 4.74 ppm with integral=(-0.4+0j)
Peak2 at 8.18 ppm with integral=(0.4+0j)
C spectrum
Peak1 @ 66.48 ppm with integral=(-2+0j)
Peak2 @ 80.19 ppm with integral=(-1.2+0j)


### Step 3.2 : Verify balanced function(f(0)!=f(1))

In [134]:
print("---------------------------------------------------")
print("P0-Uf3(00) Spectrum")
print("---------------------------------------------------")
p0_dj_uf3.h_spectrum()
p0_dj_uf3.c_spectrum()

print("\n")
print("---------------------------------------------------")
print("P0-Uf4(00) Spectrum")
print("---------------------------------------------------")
p0_dj_uf4.h_spectrum()
p0_dj_uf4.c_spectrum()

---------------------------------------------------
P0-Uf3(00) Spectrum
---------------------------------------------------
H spectrum
Peak1 at 4.74 ppm with integral=(1.6+0j)
Peak2 at 8.18 ppm with integral=(-1.6+0j)
C spectrum
Peak1 @ 66.48 ppm with integral=(1.2+0j)
Peak2 @ 80.19 ppm with integral=(-2+0j)


---------------------------------------------------
P0-Uf4(00) Spectrum
---------------------------------------------------
H spectrum
Peak1 at 4.74 ppm with integral=(1.6+0j)
Peak2 at 8.18 ppm with integral=(-1.6+0j)
C spectrum
Peak1 @ 66.48 ppm with integral=(1.2+0j)
Peak2 @ 80.19 ppm with integral=(-2+0j)


In [135]:
print("---------------------------------------------------")
print("P1-Uf3(00) Spectrum")
print("---------------------------------------------------")
p1_dj_uf3.h_spectrum()
p1_dj_uf3.c_spectrum()

print("\n")
print("---------------------------------------------------")
print("P1-Uf4(00) Spectrum")
print("---------------------------------------------------")
p1_dj_uf4.h_spectrum()
p1_dj_uf4.c_spectrum()

---------------------------------------------------
P1-Uf3(00) Spectrum
---------------------------------------------------
H spectrum
Peak1 at 4.74 ppm with integral=(2+0j)
Peak2 at 8.18 ppm with integral=(1.2+0j)
C spectrum
Peak1 @ 66.48 ppm with integral=(0.4+0j)
Peak2 @ 80.19 ppm with integral=(-0.4+0j)


---------------------------------------------------
P1-Uf4(00) Spectrum
---------------------------------------------------
H spectrum
Peak1 at 4.74 ppm with integral=(2+0j)
Peak2 at 8.18 ppm with integral=(1.2+0j)
C spectrum
Peak1 @ 66.48 ppm with integral=(0.4+0j)
Peak2 @ 80.19 ppm with integral=(-0.4+0j)


In [136]:
print("---------------------------------------------------")
print("P2-Uf3(00) Spectrum")
print("---------------------------------------------------")
p2_dj_uf3.h_spectrum()
p2_dj_uf3.c_spectrum()

print("\n")
print("---------------------------------------------------")
print("P2-Uf4(00) Spectrum")
print("---------------------------------------------------")
p2_dj_uf4.h_spectrum()
p2_dj_uf4.c_spectrum()

---------------------------------------------------
P2-Uf3(00) Spectrum
---------------------------------------------------
H spectrum
Peak1 at 4.74 ppm with integral=(0.4+0j)
Peak2 at 8.18 ppm with integral=(0.4+0j)
C spectrum
Peak1 @ 66.48 ppm with integral=(-1.6+0j)
Peak2 @ 80.19 ppm with integral=(-1.6+0j)


---------------------------------------------------
P2-Uf4(00) Spectrum
---------------------------------------------------
H spectrum
Peak1 at 4.74 ppm with integral=(0.4+0j)
Peak2 at 8.18 ppm with integral=(0.4+0j)
C spectrum
Peak1 @ 66.48 ppm with integral=(-1.6+0j)
Peak2 @ 80.19 ppm with integral=(-1.6+0j)


# (Q4) Grover's Algorithm

### Step 1: State Preparation: $H^{\otimes2} |00\rangle $
- Apply $H^{\otimes2}= H_i \otimes H_s$ on $|00\rangle$. 
- We already prepared $|00\rangle$ by temporal averaging.

### Step 2: Construct $O_3$
$$O_3 =\begin{pmatrix} 1&0&0&0\\0&1&0&0\\0&0&1&0\\0&0&0&-1 \end{pmatrix}$$

### Step 3: Construct conditional phase shift operator $P$ 
$$P=\begin{pmatrix} 1&0&0&0\\0&-1&0&0\\0&0&-1&0\\0&0&0&-1 \end{pmatrix}$$

### Step 4: Construct Grover operator: $G=H^{\otimes2}PH^{\otimes2}O$.

### Step 5: Execute Grover's algorithm 
- Execute Grovers Algorithm on P0, P1 and P2 for 1 iteration.
- Then take average to get $G H^{\otimes2}|00\rangle=|11\rangle$.

### Step 1: State Preparation: $H^{\otimes2} operator$

In [137]:
H2 = PulseSequence("H^2")\
    .add_seq(aH_i)\
    .add_seq(aH_s)\

H2.print_sequence()

H^2 Pulse Sequence: Iy(+pi/2) -> Ix(+pi/2) -> Ix(+pi/2) -> Sy(+pi/2) -> Sx(+pi/2) -> Sx(+pi/2)

In [138]:
print("H^2 upto global phase...")
np.exp(-1j*np.pi)*H2.compile().get_operator()

H^2 upto global phase...


Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 0.5  0.5  0.5  0.5]
 [ 0.5 -0.5  0.5 -0.5]
 [ 0.5  0.5 -0.5 -0.5]
 [ 0.5 -0.5 -0.5  0.5]]

## Step 2: Construct Oracles


### Step 2A: Construct $O_3$

$$O_3 =\begin{pmatrix} 1&0&0&0\\0&1&0&0\\0&0&1&0\\0&0&0&-1 \end{pmatrix}$$

In [139]:
O3 = PulseSequence("O3")\
    .add(Pulse("2IzSz(aka delay(1/2J))", IzSz, +np.pi))\
    .add(Pulse("Sy(-pi/2)", Sy, -np.pi/2))\
    .add(Pulse("Sx(+pi/2)", Sx, +np.pi/2))\
    .add(Pulse("Sy(+pi/2)", Sy, +np.pi/2))\
    .add(Pulse("Iy(-pi/2)", Iy, -np.pi/2))\
    .add(Pulse("Ix(+pi/2)", Ix, +np.pi/2))\
    .add(Pulse("Iy(+pi/2)", Iy, +np.pi/2))\
    
O3.print_sequence()

O3 Pulse Sequence: 2IzSz(aka delay(1/2J)) -> Sy(-pi/2) -> Sx(+pi/2) -> Sy(+pi/2) -> Iy(-pi/2) -> Ix(+pi/2) -> Iy(+pi/2)

In [140]:
print("O3 multiplied by global phase...")
np.exp(-1j*np.pi/4)*O3.compile().get_operator()

O3 multiplied by global phase...


Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  0. -1.]]

### Step 3: Construct conditional phase shift operator $P$ 
$$P=\begin{pmatrix} 1&0&0&0\\0&-1&0&0\\0&0&-1&0\\0&0&0&-1 \end{pmatrix}$$

In [141]:
P = PulseSequence("P")\
    .add(Pulse("2IzSz(aka delay(1/2J))", IzSz, +np.pi))\
    .add(Pulse("Sy(-pi/2)", Sy, -np.pi/2))\
    .add(Pulse("Sx(-pi/2)", Sx, -np.pi/2))\
    .add(Pulse("Sy(+pi/2)", Sy, +np.pi/2))\
    .add(Pulse("Iy(-pi/2)", Iy, -np.pi/2))\
    .add(Pulse("Ix(-pi/2)", Ix, -np.pi/2))\
    .add(Pulse("Iy(+pi/2)", Iy, +np.pi/2))\

P.print_sequence()

P Pulse Sequence: 2IzSz(aka delay(1/2J)) -> Sy(-pi/2) -> Sx(-pi/2) -> Sy(+pi/2) -> Iy(-pi/2) -> Ix(-pi/2) -> Iy(+pi/2)

In [142]:
print("P multiplied by global phase...")
np.exp(-1j*5*np.pi/4)*P.compile().get_operator()

P multiplied by global phase...


Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 1.  0.  0.  0.]
 [ 0. -1.  0.  0.]
 [ 0.  0. -1.  0.]
 [ 0.  0.  0. -1.]]

## Step 4: Construct Grover operator: $G=H^{\otimes2}PH^{\otimes2}O$.


In [143]:

def grover_iteration(Ofk):
    return PulseSequence("G")\
    .add_seq(Ofk)\
    .add_seq(H2)\
    .add_seq(P)\
    .add_seq(H2)

G = grover_iteration(O3)
G.print_sequence()

G Pulse Sequence: 2IzSz(aka delay(1/2J)) -> Sy(-pi/2) -> Sx(+pi/2) -> Sy(+pi/2) -> Iy(-pi/2) -> Ix(+pi/2) -> Iy(+pi/2) -> Iy(+pi/2) -> Ix(+pi/2) -> Ix(+pi/2) -> Sy(+pi/2) -> Sx(+pi/2) -> Sx(+pi/2) -> 2IzSz(aka delay(1/2J)) -> Sy(-pi/2) -> Sx(-pi/2) -> Sy(+pi/2) -> Iy(-pi/2) -> Ix(-pi/2) -> Iy(+pi/2) -> Iy(+pi/2) -> Ix(+pi/2) -> Ix(+pi/2) -> Sy(+pi/2) -> Sx(+pi/2) -> Sx(+pi/2)

In [144]:
G.compile().get_operator()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[0.+0.5j 0.-0.5j 0.-0.5j 0.+0.5j]
 [0.-0.5j 0.+0.5j 0.-0.5j 0.+0.5j]
 [0.-0.5j 0.-0.5j 0.+0.5j 0.+0.5j]
 [0.-0.5j 0.-0.5j 0.-0.5j 0.-0.5j]]

## Step 5: Execute Grover's algorithm 

In [145]:
def grover_algorithm(Pa: PulseSequence, Ofk: PulseSequence, iter=1):
        g = PulseSequence(f"Grover Seq -- {Pa._name} -- {Ofk._name}-- iter={iter} ::\n")\
        .add_seq(Pa)\
        .add_seq(H2)\

        for _ in range(iter):
                g_op = grover_iteration(Ofk)
                g.add_seq(g_op)
        return g

## 5A : O3 simulation

### 5A.1 O3 P0 simulation

In [146]:
g_i1_p0_o3 = grover_algorithm(p0_00, O3)
g_i1_p0_o3.print_sequence()

Grover Seq -- P0(00) -- O3-- iter=1 ::
 Pulse Sequence: Ix(pi) -> Sx(pi) -> Iy(+pi/2) -> Ix(+pi/2) -> Ix(+pi/2) -> Sy(+pi/2) -> Sx(+pi/2) -> Sx(+pi/2) -> 2IzSz(aka delay(1/2J)) -> Sy(-pi/2) -> Sx(+pi/2) -> Sy(+pi/2) -> Iy(-pi/2) -> Ix(+pi/2) -> Iy(+pi/2) -> Iy(+pi/2) -> Ix(+pi/2) -> Ix(+pi/2) -> Sy(+pi/2) -> Sx(+pi/2) -> Sx(+pi/2) -> 2IzSz(aka delay(1/2J)) -> Sy(-pi/2) -> Sx(-pi/2) -> Sy(+pi/2) -> Iy(-pi/2) -> Ix(-pi/2) -> Iy(+pi/2) -> Iy(+pi/2) -> Ix(+pi/2) -> Ix(+pi/2) -> Sy(+pi/2) -> Sx(+pi/2) -> Sx(+pi/2)

In [147]:
g_i1_p0_o3_pho = g_i1_p0_o3.compile().evolve_pho(pho_th)
g_i1_p0_o3_pho.rho

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 1.   0.   0.   0. ]
 [ 0.  -0.6  0.   0. ]
 [ 0.   0.   0.6  0. ]
 [ 0.   0.   0.  -1. ]]

In [148]:
print("Grover Algorithm :: P0 sim for O3 1 iteration")
g_i1_p0_o3_pho.h_spectrum()
g_i1_p0_o3_pho.c_spectrum()


Grover Algorithm :: P0 sim for O3 1 iteration
H spectrum
Peak1 at 4.74 ppm with integral=(0.4+0j)
Peak2 at 8.18 ppm with integral=(0.4+0j)
C spectrum
Peak1 @ 66.48 ppm with integral=(1.6+0j)
Peak2 @ 80.19 ppm with integral=(1.6+0j)


### 5A.2 O3 P1 simulation

In [149]:
g_i1_p1_o3 = grover_algorithm(p1_00, O3)
g_i1_p1_o3.print_sequence()

Grover Seq -- P1(00) -- O3-- iter=1 ::
 Pulse Sequence: Sx(+pi/2) -> 2IzSz(aka delay(1/2J)) -> Sy(+pi/2) -> Ix(+pi/2) -> 2IzSz(aka delay(1/2J)) -> Iy(+pi/2) -> Ix(pi) -> Sx(pi) -> Iy(+pi/2) -> Ix(+pi/2) -> Ix(+pi/2) -> Sy(+pi/2) -> Sx(+pi/2) -> Sx(+pi/2) -> 2IzSz(aka delay(1/2J)) -> Sy(-pi/2) -> Sx(+pi/2) -> Sy(+pi/2) -> Iy(-pi/2) -> Ix(+pi/2) -> Iy(+pi/2) -> Iy(+pi/2) -> Ix(+pi/2) -> Ix(+pi/2) -> Sy(+pi/2) -> Sx(+pi/2) -> Sx(+pi/2) -> 2IzSz(aka delay(1/2J)) -> Sy(-pi/2) -> Sx(-pi/2) -> Sy(+pi/2) -> Iy(-pi/2) -> Ix(-pi/2) -> Iy(+pi/2) -> Iy(+pi/2) -> Ix(+pi/2) -> Ix(+pi/2) -> Sy(+pi/2) -> Sx(+pi/2) -> Sx(+pi/2)

In [150]:
g_i1_p1_o3_pho = g_i1_p1_o3.compile().evolve_pho(pho_th)
g_i1_p1_o3_pho.rho

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[-0.6  0.   0.   0. ]
 [ 0.   0.6  0.   0. ]
 [ 0.   0.   1.   0. ]
 [ 0.   0.   0.  -1. ]]

In [151]:
print("Grover Algorithm :: P1 sim for O3 1 iteration")
g_i1_p1_o3_pho.h_spectrum()
g_i1_p1_o3_pho.c_spectrum()


Grover Algorithm :: P1 sim for O3 1 iteration
H spectrum
Peak1 at 4.74 ppm with integral=(-1.6+0j)
Peak2 at 8.18 ppm with integral=(1.6+0j)
C spectrum
Peak1 @ 66.48 ppm with integral=(-1.2+0j)
Peak2 @ 80.19 ppm with integral=(2+0j)


### 5A.3 O3 P2 simulation

In [152]:
g_i1_p2_o3 = grover_algorithm(p2_00, O3)
g_i1_p2_o3.print_sequence()

Grover Seq -- P2(00) -- O3-- iter=1 ::
 Pulse Sequence: Ix(+pi/2) -> 2IzSz(aka delay(1/2J)) -> Sx(+pi/2) -> Iy(+pi/2) -> 2IzSz(aka delay(1/2J)) -> Sy(+pi/2) -> Ix(pi) -> Sx(pi) -> Iy(+pi/2) -> Ix(+pi/2) -> Ix(+pi/2) -> Sy(+pi/2) -> Sx(+pi/2) -> Sx(+pi/2) -> 2IzSz(aka delay(1/2J)) -> Sy(-pi/2) -> Sx(+pi/2) -> Sy(+pi/2) -> Iy(-pi/2) -> Ix(+pi/2) -> Iy(+pi/2) -> Iy(+pi/2) -> Ix(+pi/2) -> Ix(+pi/2) -> Sy(+pi/2) -> Sx(+pi/2) -> Sx(+pi/2) -> 2IzSz(aka delay(1/2J)) -> Sy(-pi/2) -> Sx(-pi/2) -> Sy(+pi/2) -> Iy(-pi/2) -> Ix(-pi/2) -> Iy(+pi/2) -> Iy(+pi/2) -> Ix(+pi/2) -> Ix(+pi/2) -> Sy(+pi/2) -> Sx(+pi/2) -> Sx(+pi/2)

In [153]:
g_i1_p2_o3_pho = g_i1_p2_o3.compile().evolve_pho(pho_th)
g_i1_p2_o3_pho.rho

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 0.6  0.   0.   0. ]
 [ 0.   1.   0.   0. ]
 [ 0.   0.  -0.6  0. ]
 [ 0.   0.   0.  -1. ]]

In [154]:
print("Grover Algorithm :: P2 sim for O3 1 iteration")
g_i1_p2_o3_pho.h_spectrum()
g_i1_p2_o3_pho.c_spectrum()


Grover Algorithm :: P2 sim for O3 1 iteration
H spectrum
Peak1 at 4.74 ppm with integral=(1.2+0j)
Peak2 at 8.18 ppm with integral=(2+0j)
C spectrum
Peak1 @ 66.48 ppm with integral=(-0.4+0j)
Peak2 @ 80.19 ppm with integral=(0.4+0j)


### 5A.4 : Temp Averaging :: Grover algorithm O3, 1 iteration 

In [155]:
(g_i1_p0_o3_pho.rho + g_i1_p1_o3_pho.rho + g_i1_p2_o3_pho.rho ) / 3

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 0.33333333  0.          0.          0.        ]
 [ 0.          0.33333333  0.          0.        ]
 [ 0.          0.          0.33333333  0.        ]
 [ 0.          0.          0.         -1.        ]]

Clearly, this is $|11\rangle$ state.